In [2]:
!pip install pandas numpy scikit-learn joblib xgboost lightgbm catboost

  Using cached xgboost-3.0.5-py3-none-win_amd64.whl.metadata (2.1 kB)
Using cached xgboost-3.0.5-py3-none-win_amd64.whl (56.8 MB)
   ---------------------------------------- 0.0/1.5 MB ? eta -:--:--
   -------------- ------------------------- 0.5/1.5 MB 3.4 MB/s eta 0:00:01
   ------------------------------------ --- 1.3/1.5 MB 3.7 MB/s eta 0:00:01
   ---------------------------------------- 1.5/1.5 MB 3.6 MB/s  0:00:00
   ---------------------------------------- 0.0/102.4 MB ? eta -:--:--
   ---------------------------------------- 0.5/102.4 MB 3.4 MB/s eta 0:00:31
    --------------------------------------- 1.3/102.4 MB 3.9 MB/s eta 0:00:26
   - -------------------------------------- 2.6/102.4 MB 4.6 MB/s eta 0:00:22
   - -------------------------------------- 4.5/102.4 MB 5.4 MB/s eta 0:00:19
   -- ------------------------------------- 5.5/102.4 MB 5.7 MB/s eta 0:00:18
   -- ------------------------------------- 7.1/102.4 MB 5.7 MB/s eta 0:00:17
   --- ------------------------------

In [4]:
import pandas as pd
import numpy as np
import joblib
from sklearn.model_selection import GroupKFold
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.ensemble import AdaBoostClassifier, RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier

In [5]:
df = pd.read_csv("../dataset/refined_sih_dataset_ML_ready.csv")

In [6]:
for col in df.select_dtypes(include=["object"]).columns:
    le = LabelEncoder()
    df[col] = le.fit_transform(df[col].astype(str))

In [8]:
target = "rockfall_risk"
leaky_features = ["Displacement_mm", "PorePressure_kPa", "SeismicVibration_mm/s"]

X = df.drop(columns=[target] + leaky_features)
y = df[target]
groups = df["SUBDIVISION"]

# Scale features for MLP (not strictly needed for tree models)
scaler = StandardScaler()
X_scaled = pd.DataFrame(scaler.fit_transform(X), columns=X.columns)

In [10]:
models = {
    "XGBoost": XGBClassifier(
        n_estimators=200, max_depth=6, learning_rate=0.1,
        subsample=0.8, colsample_bytree=0.8,
        random_state=42, use_label_encoder=False, eval_metric="logloss"
    ),
    "LightGBM": LGBMClassifier(
        n_estimators=200, max_depth=6, learning_rate=0.1,
        subsample=0.8, colsample_bytree=0.8,
        random_state=42
    ),
    "AdaBoost": AdaBoostClassifier(
        n_estimators=200, random_state=42
    ),
    "CatBoost": CatBoostClassifier(
        iterations=200, depth=6, learning_rate=0.1,
        random_state=42, verbose=0
    ),
    "MLP": MLPClassifier(
        hidden_layer_sizes=(128, 64), activation="relu",
        solver="adam", max_iter=500, random_state=42
    ),
    "RandomForest": RandomForestClassifier(
        n_estimators=200, max_depth=10, random_state=42
    )
}

In [11]:
gkf = GroupKFold(n_splits=5)
results = {}

for model_name, model in models.items():
    print(f"\n🔹 Training {model_name}...")
    roc_auc_scores = []
    for fold, (train_idx, test_idx) in enumerate(gkf.split(X, y, groups=groups)):
        # Use scaled X only for MLP, otherwise raw X
        if model_name == "MLP":
            X_train, X_test = X_scaled.iloc[train_idx], X_scaled.iloc[test_idx]
        else:
            X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
        y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]

        model.fit(X_train, y_train)
        y_pred_prob = model.predict_proba(X_test)[:, 1]
        score = roc_auc_score(y_test, y_pred_prob)
        roc_auc_scores.append(score)
        print(f"   Fold {fold+1} ROC-AUC: {score:.4f}")

    results[model_name] = np.mean(roc_auc_scores)
    print(f"✅ {model_name} Mean ROC-AUC: {results[model_name]:.4f}")


🔹 Training XGBoost...


C:\Users\nudit\AppData\Local\Programs\Python\Python312\Lib\site-packages\xgboost\training.py:183: UserWarning: [19:43:01] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


   Fold 1 ROC-AUC: 0.8322


C:\Users\nudit\AppData\Local\Programs\Python\Python312\Lib\site-packages\xgboost\training.py:183: UserWarning: [19:43:03] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


   Fold 2 ROC-AUC: 0.8230


...

   Fold 3 ROC-AUC: 0.8145


C:\Users\nudit\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\ensemble\_weight_boosting.py:527: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(


   Fold 4 ROC-AUC: 0.8181


C:\Users\nudit\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\ensemble\_weight_boosting.py:527: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(


   Fold 5 ROC-AUC: 0.8445
✅ AdaBoost Mean ROC-AUC: 0.8186

🔹 Training CatBoost...
   Fold 1 ROC-AUC: 0.8231
   Fold 2 ROC-AUC: 0.8259
   Fold 3 ROC-AUC: 0.8507
   Fold 4 ROC-AUC: 0.8266
   Fold 5 ROC-AUC: 0.8513
✅ CatBoost Mean ROC-AUC: 0.8355

🔹 Training MLP...
   Fold 1 ROC-AUC: 0.7915
   Fold 2 ROC-AUC: 0.8221
   Fold 3 ROC-AUC: 0.7980
   Fold 4 ROC-AUC: 0.8146
   Fold 5 ROC-AUC: 0.8253
✅ MLP Mean ROC-AUC: 0.8103

🔹 Training RandomForest...
   Fold 1 ROC-AUC: 0.8254
   Fold 2 ROC-AUC: 0.8399
   Fold 3 ROC-AUC: 0.8338
   Fold 4 ROC-AUC: 0.8295
   Fold 5 ROC-AUC: 0.8476
✅ RandomForest Mean ROC-AUC: 0.8352


In [12]:
best_model_name = max(results, key=results.get)
print("\n🏆 Best Model:", best_model_name, "with ROC-AUC:", results[best_model_name])

best_model = models[best_model_name]


🏆 Best Model: CatBoost with ROC-AUC: 0.8355341179685605


In [13]:
if best_model_name == "MLP":
    best_model.fit(X_scaled, y)  # use scaled features
else:
    best_model.fit(X, y)         # use raw features

In [14]:
joblib.dump(best_model, "../rockfall_xgb_final.pkl")
print("✅ Final model saved as rockfall_xgb_final.pkl")

✅ Final model saved as rockfall_xgb_final.pkl
